# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics.distance import edit_distance
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
s1 = "ПИ19-3"
s2 = "ПМ19-3"
edit_distance(s1, s2)

1

In [3]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [4]:
mistakes = ['велечайшим', 'уменьшевшейся']

In [5]:
with open ("../data/litw-win.txt", "r", encoding='windows-1251') as fp:
    words = [line.strip().split()[1] for line in fp]

words[-10:]

['облагодетельствованным',
 'облагодетельствованных',
 'председательствовавший',
 'стадвадцатипятирублевой',
 'высокопревосходительство',
 'высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [6]:
for mistake in mistakes:
    text = text.replace(mistake, min(words, key = lambda x: edit_distance(mistake, x)))

text

'с величайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.
* *стемминг* - удаление окончания слова
* *лемматизация* - привидение слова в начальную форму

In [7]:
stemmer = SnowballStemmer('russian')

for word in word_tokenize(text):
    print(stemmer.stem(word))

с
величайш
усил
выбра
из
поток
убега
люд
кутуз
со
свит
уменьш
вдво
поеха
на
звук
выстрел
русск
оруд


In [8]:
morph = pymorphy2.MorphAnalyzer()

for word in word_tokenize(text):
    print(morph.parse(word)[0].normalized.word)

с
великий
усилие
выбраться
из
поток
убегать
человек
кутузов
с
свита
уменьшиться
вдвое
поехать
на
звук
выстрел
русский
орудие


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [9]:
text = '''Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. '''
sents = sent_tokenize(text)
sents

['Считайте слова из файла `litw-win.txt` и запишите их в список `words`.',
 'В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`.',
 'Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.']

In [10]:
cv = CountVectorizer()
cv.fit(sents)
cv.transform(sents).toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

## Лабораторная работа 9

### Расстояние редактирования

In [2]:
import pandas as pd
import random
import nltk

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [4]:
preprocessed_descriptions = pd.read_csv("../data/preprocessed_descriptions.csv", index_col=0)
preprocessed_descriptions

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


In [5]:
words = set()

for item in preprocessed_descriptions['preprocessed_descriptions']:
    if isinstance(item, str):
        words.update(word_tokenize(item))
words

{'onionsor',
 'doddle',
 'salvadorian',
 'lessoneverything',
 'nims',
 'hominy',
 'reasonably',
 'elis',
 'ad',
 'passed',
 'cuisinart',
 'uncheese',
 'chichis',
 'goosey',
 'tamarac',
 'fora',
 'miniprocessoralmost',
 'themupdate',
 'sunk',
 'vahterzoy',
 'sixpence',
 'manhattan',
 'butomgthis',
 'religions',
 'preseasoned',
 'fas',
 'restaurantparos',
 'extraveganza',
 'characteristic',
 'cakesshortbread',
 'spiffy',
 'apologetic',
 'several',
 'tells',
 'demonstrated',
 'bands',
 '83226',
 'mile',
 'healthy',
 'frankel',
 'kentaro',
 'contributions',
 'bacon438987',
 'tempting',
 'wafers',
 'ovenready',
 'guardian',
 'upstaged',
 'erawan',
 'kickpreparation',
 'tailless',
 'germits',
 'directed',
 'lowell',
 'homedemo',
 'charcoalwood',
 'much',
 'sealed',
 'caterer',
 'demo',
 'poke',
 'morels',
 'granule',
 'qualifying',
 'roomates',
 'soooooooo',
 'redford',
 'grhs',
 '263081',
 'knockout',
 'recepie',
 'james',
 'beefbroccoli',
 'itwant',
 'maturity',
 'lakes',
 'parkerhouse',
 

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [6]:
random_words = random.sample(words, 10)
random_pairs = list(zip(random_words[:5], random_words[5:]))

for pair in random_pairs:
    print(f'{pair[0]} <-> {pair[1]} = {edit_distance(pair[0], pair[1])}')

spec <-> handif = 6
13cup <-> wwwilovepicklescom = 17
onebowl <-> yummyuse = 8
brighton <-> zespri = 8
dorm <-> tooi = 3


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [7]:
from typing import Tuple

def nearest(word: str) -> Tuple[str]:
    return tuple(near_word for near_word in words if (edit_distance(word, near_word) == 1))

nearest('wafers')

('wafer', 'waters')

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
* word
* stemmed_word 
* normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для лемматизации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [9]:
stemmer = SnowballStemmer('english')
morph = pymorphy2.MorphAnalyzer()

stemmed_and_normalized_words = pd.DataFrame()
stemmed_and_normalized_words['word'] = list(words)
stemmed_and_normalized_words['stemmed_word'] = [stemmer.stem(word) for word in words]
stemmed_and_normalized_words['normalized_word'] = [morph.parse(word)[0].normalized.word for word in words]
stemmed_and_normalized_words

,word,stemmed_word,normalized_word
0,cocoa,cocoa,cocoa
1,enoughi,enoughi,enoughi
2,2425,2425,2425
3,cornfed,cornf,cornfed
4,pull,pull,pull
...,...,...,...
32863,canits,canit,canits
32864,teifi,teifi,teifi
32865,masterfoods,masterfood,masterfoods
32866,dread,dread,dread


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [10]:
all_words = list()

for item in preprocessed_descriptions['preprocessed_descriptions']:
    if isinstance(item, str):
        all_words.extend(word_tokenize(item))
all_words

['an',
 'original',
 'recipe',
 'created',
 'by',
 'chef',
 'scott',
 'meskan',
 'georges',
 'at',
 'the',
 'cove',
 'we',
 'enjoyed',
 'this',
 'when',
 'we',
 'visited',
 'this',
 'restaurant',
 'in',
 'la',
 'jolla',
 'california',
 'this',
 'recipe',
 'is',
 'requested',
 'so',
 'often',
 'they',
 'have',
 'it',
 'printed',
 'and',
 'ready',
 'at',
 'the',
 'hostess',
 'stand',
 'its',
 'unbeatable',
 'at',
 'the',
 'restaurant',
 'but',
 'i',
 'do',
 'a',
 'pretty',
 'good',
 'job',
 'at',
 'home',
 'too',
 'if',
 'i',
 'do',
 'say',
 'so',
 'myself',
 'my',
 'children',
 'and',
 'their',
 'friends',
 'ask',
 'for',
 'my',
 'homemade',
 'popsicles',
 'morning',
 'noon',
 'and',
 'night',
 'i',
 'never',
 'turn',
 'them',
 'down',
 'who',
 'am',
 'i',
 'to',
 'tell',
 'them',
 'that',
 'they',
 'are',
 'good',
 'for',
 'them',
 'for',
 'variety',
 'i',
 'substitute',
 'different',
 'flavours',
 'of',
 'frozen',
 'juice',
 'grape',
 'fruit',
 'punch',
 'tropical',
 'etc',
 'these',


In [ ]:
clean_all_words = [word for word in all_words if word not in nltk.corpus.stopwords.words()]

In [ ]:
(len(all_words) - len(clean_all_words))/len(all_words)

In [ ]:
print("ДО УДАЛЕНИЯ")
for word, number in nltk.FreqDist(all_words).most_common(10):
    print(f"{word} x {number}")

In [ ]:
print("ПОСЛЕ УДАЛЕНИЯ")
for word, number in nltk.FreqDist(clean_all_words).most_common(10): 
    print(f"{word} x {number}")

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
random_recipes = random.sample(list(preprocessed_descriptions.index), 5)
random_recipes = {recipe_index: preprocessed_descriptions['preprocessed_descriptions'].loc[recipe_index] for recipe_index in random_recipes}
random_recipes

{14826: 'posted by request can be used either as a salad dressing or a dip for tortilla chips or vegetables',
 18920: 'this is a yummy and healthy drink that the kids really enjoy',
 467: 'this recipe is easy to double or cut in half so its great for summer picnics or quiet dinners recipe is from simple  delicious',
 10139: 'an easy and delicious way to have summer peaches which makes a great cooking project for kids when they have nothing to do recipe adapted from bon appetit august 1981',
 13151: 'guy my husband is from oklahoma i am a native californian who never tasted okra until we married i introduced him to the finer points of serrano chilies and in return he created this delicious soup recipe that combines both okra and serranos'}

In [32]:
vectorizer = TfidfVectorizer()
recipe_vectors = vectorizer.fit_transform(random_recipes.values()).toarray()
recipe_vectors

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.24216427, 0.        , 0.24216427, 0.        ,
        0.        , 0.24216427, 0.        , 0.24216427, 0.        ,
        0.24216427, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.24216427, 0.        , 0.        ,
        0.24216427, 0.        , 0.        , 0.24216427, 0.        ,
        0.        , 0.16218024, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39075322, 0.        , 0.        , 0.        , 0.24216427,
        0.        , 0.        , 0.        , 0.        , 0.24216427,
        0.        , 0.24216427, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [33]:
import scipy

proximity_of_recipes = pd.DataFrame()

for vector in recipe_vectors:
    proximity_of_recipes[str(vector)] = [scipy.spatial.distance.cosine(vector, other_vector) for other_vector in recipe_vectors]

proximity_of_recipes.columns = list(random_recipes.keys())
proximity_of_recipes.index = list(random_recipes.keys())
proximity_of_recipes

,14826,18920,467,10139,13151
14826,0.000000,1.000000,0.834883,0.978957,1.000000
18920,1.000000,0.000000,0.895654,0.926056,0.824355
467,0.834883,0.895654,0.000000,0.779595,0.845231
10139,0.978957,0.926056,0.779595,0.000000,0.900586
13151,1.000000,0.824355,0.845231,0.900586,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Наиболее похожими являются рецепты, косинусное расстояние между которыми ближе всего к 0, т.к. значение между рецептом самим с собой = 0

(Т. к. рецепты выбирали рандомные, то номера будут изменяться после перезапуска кода)